# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.script import *

In [ ]:
#| export
@call_parse
def add_one(
    num: int # first number
    ) -> int: # result
    result = num + 1
    print(result)
    return result

In [ ]:
assert add_one(2) == 3